#1. Indexing

In [ ]:
!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier

  Cloning https://github.com/terrier-org/pyterrier.git to /tmp/pip-install-15kj1zos/python-terrier
  Running command git clone -q https://github.com/terrier-org/pyterrier.git /tmp/pip-install-15kj1zos/python-terrier
     |████████████████████████████████| 1.1MB 4.1MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.3.0.dev0-cp36-none-any.whl size=41247 sha256=91a5dd3b989ef5e99df568af8090f50c23c6ff6573fa45aa6d87f0e1dd3922ae
  Stored in directory: /tmp/pip-ephem-wheel-cache-1vn7nrxa/wheels/cc/bb/69/836d846a92c787b35ca6478119c0033762ab2b95d866eeb288
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=cf7cdbafc24d5a1365985b505accf6c26b7907cb5b64cc7a51f339d297af82da
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
  Created wheel for pytrec-eval: filename=pytrec_eval-0.4-cp36-cp36m-linux_x86_64.whl size=273871 sha256=59c302ea477890b6e60d7

In [ ]:
import pyterrier as pt
if not pt.started():
  pt.init()

terrier-assemblies 5.3  jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.4  jar not found, downloading to /root/.pyterrier...
Done


Get TREC docs

In [ ]:
# dataset = pt.datasets.get_dataset("vaswani")

# print("Files in vaswani corpus: %s " % dataset.get_corpus())

In [ ]:
index_path = "./index"
!rm -rf ./index
indexer = pt.TRECCollectionIndexer(index_path)

In [ ]:
index_properies = {"block.indexing":"true", "invertedfile.lexiconscanner":"pointers", "FieldTags.process":"TITLE,PAPERABSTRACT,ELSE", "indexer.meta.forward.keys":"docno,title,paperabstract", "indexer.meta.forward.keylens":"42,200,1024"}
indexer.setProperties(**index_properies)

# indexref = indexer.index(dataset.get_corpus())


In [ ]:
indexref = indexer.index("/content/drive/My Drive/Project/trecdocs-file.trec")

In [ ]:
index = pt.IndexFactory.of(indexref)
print(index.getCollectionStatistics().toString())

Number of documents: 8879
Number of terms: 90495
Number of fields: 3
Number of tokens: 1435512
Field names: [TITLE, PAPERABSTRACT, ELSE]
Positions:   false



#2. Retrieve

In [ ]:
pt.BatchRetrieve(indexref,controls={"wmodel":"DPH"}).transform("cloud")

,qid,docid,docno,rank,score,query
0,1,3854,6784d4cb2da46e7661d21f4388830a231f2781f4,0,6.105648,cloud
1,1,7507,b2fdee22aa02477292b858fbafcb418932732bce,1,6.096576,cloud
2,1,8095,78da12073562af63b2f0e5f22f2503583e485702,2,6.072798,cloud
3,1,1725,8fc928bb430d3f72ac876ca156042ad1860acacd,3,6.045313,cloud
4,1,4039,4da704492963958f3242ce4553786195d427c41f,4,5.978381,cloud
...,...,...,...,...,...,...
140,1,2631,03c7355211f80f7e415fdeb544520776a3d4b0d1,140,2.155709,cloud
141,1,3124,2228b4208c5ea6754df6edcae805038f3e47857c,141,2.003419,cloud
142,1,8813,729d8359a90d5bb66fa9efcd0daf9e7434bfbe97,142,1.912169,cloud
143,1,1911,2e32718bd9174768782973c0c80f52d6a06b148c,143,1.498746,cloud


#3. Evaluation

In [ ]:
TF_IDF = pt.BatchRetrieve(indexref, controls = {"wmodel": "TF_IDF"})
BM25 = pt.BatchRetrieve(indexref, controls = {"wmodel": "BM25"})
PL2 = pt.BatchRetrieve(indexref, controls = {"wmodel": "PL2"})
WeightingModel = pt.BatchRetrieve(indexref, controls = {"wmodel": "WeightingModel"})
ML2 = pt.BatchRetrieve(indexref, controls = {"wmodel": "ML2"})


In [ ]:
import pytrec_eval
pytrec_eval.supported_measures

{'11pt_avg',
 'G',
 'P',
 'Rndcg',
 'Rprec',
 'Rprec_mult',
 'binG',
 'bpref',
 'gm_bpref',
 'gm_map',
 'infAP',
 'iprec_at_recall',
 'map',
 'map_cut',
 'ndcg',
 'ndcg_cut',
 'ndcg_rel',
 'num_nonrel_judged_ret',
 'num_q',
 'num_rel',
 'num_rel_ret',
 'num_ret',
 'recall',
 'recip_rank',
 'relative_P',
 'relstring',
 'runid',
 'set_F',
 'set_P',
 'set_map',
 'set_recall',
 'set_relative_P',
 'success',
 'utility'}

In [ ]:
eval = pt.pipelines.Experiment(
    [TF_IDF,BM25,ML2,PL2],
    "/content/drive/My Drive/Project/topic",
    "/content/drive/My Drive/Project/qrel",
    ['G','map','ndcg'])
eval

,name,G,map,ndcg
0,BR(TF_IDF),0.393483,0.318778,0.529667
1,BR(BM25),0.392165,0.317333,0.528754
2,BR(ML2),0.386033,0.306567,0.519826
3,BR(PL2),0.385581,0.308104,0.521119


Doesn't consider fairness. So each metric is very low.